# Travelling Salesperson Problem solved using genetic algorithms

In [1]:
# Imports 
import numpy as np
import random

np.random.seed(42)

from datetime import datetime

In [2]:
# Parameters
n_cities = 20

n_population = 100

mutation_rate = 0.3

In [3]:
# Generating a list of coordenades representing each city
coordinates_list = [[x,y] for x,y in zip(np.random.randint(0,100,n_cities),np.random.randint(0,100,n_cities))]
names_list = np.array(['Berlin', 'London', 'Moscow', 'Barcelona', 'Rome', 'Paris', 'Vienna', 'Munich', 'Istanbul', 'Kyiv', 'Bucharest', 'Minsk', 'Warsaw', 'Budapest', 'Milan', 'Prague', 'Sofia', 'Birmingham', 'Brussels', 'Amsterdam'])
cities_dict = { x:y for x,y in zip(names_list,coordinates_list)}

# Function to compute the distance between two points
def compute_city_distance_coordinates(a,b):
    return ((a[0]-b[0])**2+(a[1]-b[1])**2)**0.5

def compute_city_distance_names(city_a, city_b, cities_dict):
    return compute_city_distance_coordinates(cities_dict[city_a], cities_dict[city_b])

cities_dict

{'Berlin': [51, 1],
 'London': [92, 63],
 'Moscow': [14, 59],
 'Barcelona': [71, 20],
 'Rome': [60, 32],
 'Paris': [20, 75],
 'Vienna': [82, 57],
 'Munich': [86, 21],
 'Istanbul': [74, 88],
 'Kyiv': [74, 48],
 'Bucharest': [87, 90],
 'Minsk': [99, 58],
 'Warsaw': [23, 41],
 'Budapest': [2, 91],
 'Milan': [21, 59],
 'Prague': [52, 79],
 'Sofia': [1, 14],
 'Birmingham': [87, 61],
 'Brussels': [29, 61],
 'Amsterdam': [37, 46]}

## 1. Create the first population set
We randomly shuffle the cities N times where N=population_size

In [4]:
# First step: Create the first population set
def genesis(city_list, n_population):

    population_set = []
    for i in range(n_population):
        #Randomly generating a new solution
        sol_i = city_list[np.random.choice(list(range(n_cities)), n_cities, replace=False)]
        population_set.append(sol_i)
    return np.array(population_set)

population_set = genesis(names_list, n_population)
population_set

array([['Prague', 'Munich', 'Budapest', ..., 'Moscow', 'Minsk',
        'Brussels'],
       ['Budapest', 'Prague', 'Rome', ..., 'Vienna', 'Milan', 'London'],
       ['Moscow', 'Berlin', 'Paris', ..., 'London', 'Minsk',
        'Birmingham'],
       ...,
       ['Budapest', 'Berlin', 'Milan', ..., 'Istanbul', 'Paris',
        'Moscow'],
       ['Bucharest', 'Warsaw', 'Minsk', ..., 'Barcelona', 'Prague',
        'Amsterdam'],
       ['Prague', 'Minsk', 'Berlin', ..., 'Barcelona', 'Budapest',
        'Brussels']], dtype='<U10')

## 2. Evaluate solutions fitness
The solutions are defined so that the first element on the list is the first city to visit, then the second, etc. and the last city is linked to the first.
The fitness function needs to compute the distance between subsequent cities.

In [5]:
def fitness_eval(city_list, cities_dict):
    total = 0
    for i in range(n_cities-1):
        a = city_list[i]
        b = city_list[i+1]
        total += compute_city_distance_names(a,b, cities_dict)
    return total

In [6]:
def get_all_fitnes(population_set, cities_dict):
    fitnes_list = np.zeros(len(population_set))

    #Looping over all solutions computing the fitness for each solution
    for i in  range(len(population_set)):
        fitnes_list[i] = fitness_eval(population_set[i], cities_dict)

    return fitnes_list

fitnes_list = get_all_fitnes(population_set,cities_dict)
fitnes_list

array([1255.15287164, 1052.53607429,  965.51789891, 1036.36183177,
        879.44027804, 1036.5560631 ,  937.30697091,  894.68193131,
       1030.58421856, 1050.19970288, 1020.39604193, 1033.71475965,
        903.75660497, 1147.30725354, 1124.82980456, 1008.61780339,
        884.09179724, 1157.98023232,  947.90741993, 1019.47661627,
        997.10745475,  912.05976107,  902.30648574, 1029.85026604,
       1113.56434666,  838.92461468, 1003.97594363, 1003.49502143,
        955.55361584,  979.03611426,  897.86346402, 1023.28164421,
        999.59478622, 1076.50742562,  956.85293143,  908.07173724,
        968.7582608 , 1080.75558082, 1015.45390227, 1018.88283913,
        983.38406618, 1170.33698574, 1025.03223048,  981.16514434,
       1004.66637234,  957.60798338,  945.20208566,  989.63711937,
       1055.07742869,  906.24499451, 1028.06717304,  977.70047293,
       1035.42410739,  831.49118547, 1086.49202813, 1092.85512369,
        971.01347048,  985.35653284,  990.59069671,  957.52755

# 3. Progenitors selection
I will select a new set of progenitors using the Roulette Wheel Selection. Generates a list of progenitor pairs where N= len(population_set) but at each position there are two solutions to merge

In [7]:
# ORG
def progenitor_selection(population_set,fitnes_list):
    total_fit = fitnes_list.sum()
    prob_list = fitnes_list/total_fit
    
    #Notice there is the chance that a progenitor. mates with oneself
    progenitor_list_a = np.random.choice(list(range(len(population_set))), len(population_set),p=prob_list, replace=True)
    progenitor_list_b = np.random.choice(list(range(len(population_set))), len(population_set),p=prob_list, replace=True)
    
    progenitor_list_a = population_set[progenitor_list_a]
    progenitor_list_b = population_set[progenitor_list_b]
    
    
    return np.array([progenitor_list_a,progenitor_list_b])


progenitor_list = progenitor_selection(population_set,fitnes_list)
progenitor_list[0][2]

array(['Paris', 'Birmingham', 'Barcelona', 'Bucharest', 'Warsaw',
       'Prague', 'Amsterdam', 'Rome', 'Brussels', 'Munich', 'Istanbul',
       'Milan', 'Moscow', 'London', 'Berlin', 'Vienna', 'Budapest',
       'Kyiv', 'Sofia', 'Minsk'], dtype='<U10')

In [8]:


def progenitor_selection_by_rank(population_set, fitnes_list, n_pairs):
    n = len(population_set)
    rank_sum = n * (n + 1) / 2
    prob_list = [i/rank_sum for i in range(1,101)][::-1]

    f = fitnes_list.argsort()
    
    #Notice there is the chance that a progenitor. mates with oneself
    progenitor_list_a = np.random.choice(f, n_pairs, p=prob_list, replace=True)
    progenitor_list_b = np.random.choice(f, n_pairs, p=prob_list, replace=True)
    
    progenitor_list_a = population_set[progenitor_list_a]
    progenitor_list_b = population_set[progenitor_list_b]
    
    
    return np.array([progenitor_list_a,progenitor_list_b])


progenitor_list = progenitor_selection_by_rank(population_set,fitnes_list, len(population_set))
progenitor_list[0][2]

array(['Rome', 'Barcelona', 'Birmingham', 'Kyiv', 'Brussels', 'Amsterdam',
       'Paris', 'Istanbul', 'Milan', 'Bucharest', 'London', 'Vienna',
       'Moscow', 'Minsk', 'Sofia', 'Prague', 'Budapest', 'Munich',
       'Berlin', 'Warsaw'], dtype='<U10')

# 4. Mating
For each pair of  parents we'll generate an offspring pair. Since we cannot repeat cities what we'll do is copy a random chunk from one progenitor and fill the blanks with the other progenitor.

In [9]:
def mate_progenitors(prog_a, prog_b):
    offspring = prog_a[0:5]

    for city in prog_b:

        if not city in offspring:
            offspring = np.concatenate((offspring,[city]))

    return offspring
            
    
    
def mate_population(progenitor_list):
    new_population_set = []
    for i in range(progenitor_list.shape[1]):
        prog_a, prog_b = progenitor_list[0][i], progenitor_list[1][i]
        offspring = mate_progenitors(prog_a, prog_b)
        new_population_set.append(offspring)
        
    return new_population_set

new_population_set = mate_population(progenitor_list)
new_population_set[0]

array(['Warsaw', 'Amsterdam', 'London', 'Paris', 'Munich', 'Moscow',
       'Budapest', 'Istanbul', 'Prague', 'Barcelona', 'Vienna',
       'Brussels', 'Berlin', 'Rome', 'Minsk', 'Bucharest', 'Birmingham',
       'Milan', 'Kyiv', 'Sofia'], dtype='<U10')

# 5. Mutation
Now for each element of the new population we add a random chance of swapping

In [10]:
def mutate_offspring(offspring):
    for q in range(int(n_cities*mutation_rate)):
        a = np.random.randint(0,n_cities)
        b = np.random.randint(0,n_cities)

        offspring[a], offspring[b] = offspring[b], offspring[a]

    return offspring
    
    
def mutate_population(new_population_set):
    mutated_pop = []
    for offspring in new_population_set:
        mutated_pop.append(mutate_offspring(offspring))
    return mutated_pop

mutated_pop = mutate_population(new_population_set)
mutated_pop[0]

array(['Warsaw', 'Rome', 'Sofia', 'Paris', 'Amsterdam', 'Minsk',
       'Barcelona', 'Istanbul', 'Prague', 'Munich', 'Vienna', 'Brussels',
       'Berlin', 'Budapest', 'Moscow', 'Bucharest', 'Birmingham', 'Milan',
       'Kyiv', 'London'], dtype='<U10')

# 6. Stopping
To select the stopping criteria we'll need to create a loop to stop first. Then I'll set it to loop at 1000 iterations.

In [11]:
best_solution = [-1,np.inf,np.array([])]

n_elitism=1
mutation_rate = 0.3

n_gens = 10000

for i in range(n_gens):
    if i%100==0: print(i, fitnes_list.min(), fitnes_list.mean(), datetime.now().strftime("%d/%m/%y %H:%M"))
    
    
    if i == 0:
        #Compute all fitness
        fitnes_list = get_all_fitnes(mutated_pop,cities_dict)
    else:
        #Compute new fitness
        fitnes_list2 = get_all_fitnes(mutated_pop2,cities_dict)
        fitnes_list = np.concatenate((fitnes_list[fitnes_list.argsort()][:n_elitism],fitnes_list2))
    
    #Saving the best solution
    if fitnes_list.min() < best_solution[1]:
        best_solution[0] = i
        best_solution[1] = fitnes_list.min()
        best_solution[2] = np.array(mutated_pop)[fitnes_list.min() == fitnes_list]
    
    #This can be set as progressive elitism
    n_elitism= min(len(new_population_set), int(np.log(i+1)))
    mutation_rate = i/n_gens
    
    progenitor_list = progenitor_selection_by_rank(population_set,fitnes_list, len(population_set)- n_elitism)
    offspring = mate_population(progenitor_list)
    
    mutated_pop2 = mutate_population(offspring)
    
    mutated_pop = np.concatenate((np.array(mutated_pop)[fitnes_list.argsort()[:n_elitism]],mutated_pop2[:len(population_set)-n_elitism]))
    


0 831.4911854722953 1012.7187535355878 02/02/21 10:41
100 661.4915724246024 993.6564923658315 02/02/21 10:41
200 661.4915724246024 980.0747835891316 02/02/21 10:41
300 661.4915724246024 981.5789070068079 02/02/21 10:41
400 661.4915724246024 1001.5336241903277 02/02/21 10:41
500 661.4915724246024 993.5384419464183 02/02/21 10:41
600 646.3105150486459 984.7515142534193 02/02/21 10:41
700 646.3105150486459 998.1302579638342 02/02/21 10:41
800 638.9280599096186 990.7301196348099 02/02/21 10:41
900 638.9280599096186 994.3061309607273 02/02/21 10:42
1000 638.9280599096186 992.9901610369521 02/02/21 10:42
1100 638.9280599096186 979.0263126806727 02/02/21 10:42
1200 638.9280599096186 985.1031180080256 02/02/21 10:42
1300 638.9280599096186 976.9662498668615 02/02/21 10:42
1400 638.9280599096186 985.4593800635975 02/02/21 10:42
1500 638.9280599096186 988.3946599606028 02/02/21 10:42
1600 638.9280599096186 979.7106617224927 02/02/21 10:42
1700 638.9280599096186 995.3736670785231 02/02/21 10:42
18

In [12]:
#645

In [13]:
np.log(5000+1)**0.5

2.918457327325329